<a href="https://colab.research.google.com/github/i-supermario/Fellowship.ai/blob/main/STL10_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import copy

plt.ion()   

In [18]:
transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((272,272)),
        torchvision.transforms.RandomRotation(15,),
        torchvision.transforms.RandomCrop(256),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor()])
dataset = torchvision.datasets.STL10(root="/content/STL10",split='train',transform=transforms,download=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified


In [19]:
train_set, val_set = torch.utils.data.random_split(dataset, [4000, 1000])
image_datasets = {'train': train_set,'val':val_set}

In [21]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [22]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    for phase in ['train','val']:
      if phase == 'train':
        model.train()  # Set model to training mode
      else:
        model.eval()   # Set model to evaluate mode

      running_loss = 0.0
      running_corrects = 0
      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase=='train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)
          if phase == 'train':
            loss.backward()              
            optimizer.step()
        

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model


In [24]:
model_ft = torchvision.models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, len(dataset.classes))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [25]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6804 Acc: 0.7835
val Loss: 0.2623 Acc: 0.9140
Epoch 1/24
----------
train Loss: 0.2989 Acc: 0.9020
val Loss: 0.2458 Acc: 0.9130
Epoch 2/24
----------
train Loss: 0.2262 Acc: 0.9297
val Loss: 0.1779 Acc: 0.9400
Epoch 3/24
----------
train Loss: 0.1638 Acc: 0.9483
val Loss: 0.1683 Acc: 0.9380
Epoch 4/24
----------
train Loss: 0.1349 Acc: 0.9567
val Loss: 0.2016 Acc: 0.9360
Epoch 5/24
----------
train Loss: 0.1106 Acc: 0.9655
val Loss: 0.1826 Acc: 0.9350
Epoch 6/24
----------
train Loss: 0.0956 Acc: 0.9713
val Loss: 0.2122 Acc: 0.9360
Epoch 7/24
----------
train Loss: 0.0669 Acc: 0.9815
val Loss: 0.1405 Acc: 0.9560
Epoch 8/24
----------
train Loss: 0.0543 Acc: 0.9833
val Loss: 0.1544 Acc: 0.9450
Epoch 9/24
----------
train Loss: 0.0505 Acc: 0.9895
val Loss: 0.1373 Acc: 0.9540
Epoch 10/24
----------
train Loss: 0.0463 Acc: 0.9888
val Loss: 0.1240 Acc: 0.9570
Epoch 11/24
----------
train Loss: 0.0444 Acc: 0.9898
val Loss: 0.1340 Acc: 0.9580
Epoch 12/24
--

In [29]:
testdata = torchvision.datasets.STL10(root="/content/STL10",split='test',transform=transforms,download=True)
testloader = torch.utils.data.DataLoader(testdata, batch_size=10,shuffle=True)

model_ft.eval()
running_loss = 0.0
running_corrects = 0
for inputs, labels in testloader:
  inputs = inputs.to(device)
  labels = labels.to(device)

  with torch.set_grad_enabled(False):
    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)
  
  running_loss += loss.item() * inputs.size(0)
  running_corrects += torch.sum(preds == labels.data)

avg_loss = running_loss / len(testdata)
avg_acc = running_corrects.double() / len(testdata)
          
print('Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

Files already downloaded and verified
Loss: 0.1416 Acc: 0.9535
